In [ ]:
import os
import tensorflow as tf
import tensorflow.keras as K
import numpy as np
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'


In [ ]:
# to fix issue with saving keras applications
# K.learning_phase = K.backend.learning_phase

_, (X, Y) = K.datasets.cifar10.load_data()
def preprocess_data(X, Y):
    """function pre_data"""
    X = X / 255.0
    Y = K.utils.to_categorical(Y, 10)
    return X, Y
X_p, Y_p = preprocess_data(X, Y)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [ ]:
(x_train, y_train), (x_test, y_test) = K.datasets.cifar10.load_data()
x_train, y_train = preprocess_data(x_train, y_train)
x_test, y_test = preprocess_data(x_test, y_test)
base_model = K.applications.DenseNet121(weights='imagenet', include_top=False,
                                     input_shape=(224, 224, 3))
base_model.trainable = False
# Define the model structure
inputs = K.Input(shape=(32, 32, 3))
input = K.layers.Lambda(lambda image: tf.image.resize(image, (224, 224)))(inputs)
x = base_model(input, training=False)
x = K.layers.GlobalAveragePooling2D()(x)
x = K.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = K.layers.Dropout(0.3)(x)
x = K.layers.BatchNormalization()(x)
x = K.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = K.layers.Dropout(0.3)(x)
x = K.layers.BatchNormalization()(x)
outputs = K.layers.Dense(10, activation='softmax')(x)
model = K.Model(inputs, outputs)
model.compile(optimizer=K.optimizers.Adam(),
              loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=3, batch_size=500, verbose=1, validation_data=(x_test, y_test))
model.save('tmp_cifar10.h5')

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/3
100/100 ━━━━━━━━━━━━━━━━━━━━ 240s 1s/step - accuracy: 0.4608 - loss: 1.8769 - val_accuracy: 0.7718 - val_loss: 0.9350
Epoch 2/3
100/100 ━━━━━━━━━━━━━━━━━━━━ 156s 1s/step - accuracy: 0.7486 - loss: 0.9467 - val_accuracy: 0.8040 - val_loss: 0.7651
Epoch 3/3
100/100 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.7854 - loss: 0.8111 - val_accuracy: 0.8162 - val_loss: 0.6989


In [ ]:
print(os.getcwd())

/content


In [ ]:
os.listdir('/content')

['.config', 'tmp_cifar10.h5', 'sample_data']

In [ ]:
from google.colab import files
files.download('tmp_cifar10.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model.evaluate(X_p, Y_p, batch_size=128, verbose=1)

79/79 ━━━━━━━━━━━━━━━━━━━━ 56s 377ms/step - accuracy: 0.8119 - loss: 0.7013


[0.698909342288971, 0.8162000179290771]

In [ ]:
print(f"Total layers in base_model: {len(base_model.layers)}")

Total layers in base_model: 427


In [ ]:
# Unfreeze some layers for fine-tuning
base_model.trainable = True

# Partial Unfreeze: Only unfreezing the last 122 layers
for layer in base_model.layers[:-122]:
    layer.trainable = False

# Re-compile the model with a lower learning rate for fine-tuning
model.compile(optimizer=K.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
model.fit(x_train, y_train, epochs=5, batch_size=128, verbose=1, validation_data=(x_test, y_test))

# Evaluate the fine-tuned model
model.evaluate(x_test, y_test, batch_size=128, verbose=1)
model.save('fine_tuned_cifar10.h5')

Epoch 1/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 262s 505ms/step - accuracy: 0.7461 - loss: 0.9178 - val_accuracy: 0.8261 - val_loss: 0.6523
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 178s 371ms/step - accuracy: 0.8102 - loss: 0.7318 - val_accuracy: 0.8451 - val_loss: 0.6033
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 201s 370ms/step - accuracy: 0.8340 - loss: 0.6581 - val_accuracy: 0.8555 - val_loss: 0.5697
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 202s 371ms/step - accuracy: 0.8500 - loss: 0.6129 - val_accuracy: 0.8639 - val_loss: 0.5444
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 202s 372ms/step - accuracy: 0.8645 - loss: 0.5674 - val_accuracy: 0.8709 - val_loss: 0.5240
79/79 ━━━━━━━━━━━━━━━━━━━━ 20s 252ms/step - accuracy: 0.8707 - loss: 0.5230


In [ ]:
os.listdir('/content')

['.config', 'tmp_cifar10.h5', 'fine_tuned_cifar10.h5', 'sample_data']

In [ ]:
files.download('fine_tuned_cifar10.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(f"Total layers in the fine-tuned model: {len(model.layers)}")

Total layers in the fine-tuned model: 11


In [ ]:
print(f"Total layers in base_model: {len(base_model.layers)}")

Total layers in base_model: 427
